**Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Train a classifier on
this new training set. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s pre‐
dictions. How does it compare to the voting classifier you trained earlier?**

### Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version= 1, as_frame= False)

from sklearn.model_selection import train_test_split

X_val, X_test, y_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_val, y_val, test_size=10000, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

models = [
          RandomForestClassifier(n_estimators=100, n_jobs=-1),
          ExtraTreesClassifier(n_estimators=100, n_jobs=-1),
          SVC(n_jobs=-1)
]

for model in models:
  print("Training", model,"...")
  model.fit(X_train, y_train)

Training RandomForestClassifier(n_jobs=-1) ...
Training ExtraTreesClassifier(n_jobs=-1) ...
Training SVC() ...


In [4]:
predictions = [model.predict(X_val) for model in models]

In [11]:
predictions

[array(['5', '8', '2', ..., '7', '6', '7'], dtype=object),
 array(['5', '8', '2', ..., '7', '6', '7'], dtype=object),
 array(['5', '8', '2', ..., '7', '6', '7'], dtype=object)]

Each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. 

In [22]:
instances_pred = [ [predictions[0][i], predictions[1][i], predictions[2][i] ] 
                    for i in range(len(predictions[0]))]
instances_pred[:3]

[['5', '5', '5'], ['8', '8', '8'], ['2', '2', '2']]

In [33]:
new_train_set = [[instances_pred[i], y_val[i]] for i in range(len(y_val))]
new_train_set[:5]

[[['5', '5', '5'], '5'],
 [['8', '8', '8'], '8'],
 [['2', '2', '2'], '2'],
 [['4', '4', '4'], '4'],
 [['4', '4', '4'], '4']]

### Train a classifier on this new training set

In [36]:
blender = RandomForestClassifier(n_estimators=200, oob_score=True)
blender.fit(instances_pred, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True)

In [40]:
blender.oob_score_

0.9763

### Now let’s evaluate the ensemble on the test set.

#### For each image in the test set, make predictions with all your classifiers
Then feed the predictions to the blender to get the ensemble’s pre‐
dictions. How does it compare to the voting classifier you trained earlier?

In [44]:
from sklearn.metrics import accuracy_score

def blender_model(models, X_test, y_test, blender):
  predictions = [model.predict(X_test) for model in models]
  instances_pred = [ [predictions[0][i], predictions[1][i], predictions[2][i] ] 
                    for i in range(len(predictions[0]))]
  y_pred = blender.predict(instances_pred)
  print(accuracy_score(y_test, y_pred))

In [45]:
blender_model(models, X_test, y_test, blender)

0.9722


97.22% of accuracy. The voting classifier with same algorithms reached 97.02%, which means this stacking ensemble is only 0.2% better than the former, even though the computing resources required are very larger.